# Traducción de frases en ingles a enunciados de lógica proposicional

En un curso de Lógica, un ejercicio consiste en convertir una frase  como ésta:

> *Sieglinde sobrevivirá, y o bien su hijo obtendrá el Anillo y el plan de Wotan se cumplirá o bien el Valhalla será destruido.*

junto con las definiciones de las proposiciones:

    P: Sieglinde sobrevivirá
    P: El hijo de Sieglinde obtendrá el Anillo
    R: El plan de Wotan se cumplirá
    S: El Valhalla será destruido

en un enunciado formal de lógica proposicional:

    P ⋀ ((Q ⋀ R) ∨ S)
    
En el caso de algunas frases, se necesitan conocimientos detallados para conseguir una buena traducción. Las dos frases siguientes son ambiguas, con diferentes interpretaciones preferidas, y traducirlas correctamente requiere conocimientos sobre hábitos alimenticios:

    Comeré ensalada o comeré pan y comeré mantequilla.     P ∨ (Q ⋀ R)
    Comeré ensalada o comeré sopa y comeré helado.        (P ∨ Q) ⋀ R

Pero para muchas frases, el proceso de traducción es automático, sin necesidad de conocimientos especiales.  

Desarrollar un programa es importante para manejar estas frases fáciles. El programa se basa en la idea de una serie de reglas de traducción de la forma:

    Regla('{P} ⇒ {Q}', 'si {P} entonces {Q}', 'si {P}, {Q}')
    
lo que significa que la traducción lógica tendrá la forma `'P ⇒ Q'`, siempre que la frase tenga la forma `'si P entonces Q'` o  `'si P, Q'`, donde `P` y `Q` puede coincidir con cualquier sucesión no vacía de caracteres.  Lo que coincida con `P` y `Q` serán procesados recursivamente por las reglas. Las reglas están en ordenserán procesados recursivamente por las reglas. Las reglas están en orden&mdash;de arriba abajo, de izquierda a derecha, y la primera regla que coincida en ese orden será aceptada, sin importar qué, así que asegúrate de ordenar tus reglas cuidadosamente. Una pauta que he seguido es poner todas las reglas que empiezan con una palabra clave (como `'si'` o `'tampoco'`.) antes de las reglas que empiezan por una variable (como `'{P}'`); Así evitará que una palabra clave quede engullida accidentalmente dentro de un `'{P}'`.

Considere la frase de ejemplo `"Si amarte está mal, no quiero tener razón."` Esto debería coincidir con el patrón `'si {P}, {Q}'` with the variable `P` equal to `"amarte está mal"`. Pero no quiero que la variable `Q` sea
`"No quiero tener razón"`, más bien, quiero tener `～Q` igual a `"Quiero tener razón"`. Así que además de tener un conjunto de `Reglas` para manejar el `'si {P}, {Q}'` también tendré una lista de "negaciones" para manejar "no" y similares.

Aquí está el código para procesar las definiciones `Reglas` (usando [expresiones regulares](https://docs.python.org/3.5/library/re.html) y aqui [  otra informacion](https://devarea.com/python-regular-expressions-practical-guide/), para aclarar lo que a veces puede resultar confuso).

In [66]:
import re

def Rule(output, *patterns):
    "Una regla que produce `salida` si toda la entrada coincide con alguno de los `patrones`."
    return (output, [nombre_grupo(pat) + '$' for pat in patterns])

def nombre_grupo(pat):
    "Sustituir '{Q}' con '(?P<Q>.+?)', lo que significa 'coincide con 1 o más caracteres, y lo llama Q'"
    return re.sub('{(.)}', r'(?P<\1>.+?)', pat)

def palabra(w):
    "Devuelve una expresión regular que coincide con w como una palabra completa (no letras dentro de una palabra)."
    return r'\b' + w + r'\b' # '\b' coincide con el límite de la palabra

Veamos cómo es una regla:

In [ ]:
Rule('{P} ⇒ {Q}', 'if {P} then {Q}', 'if {P}, {Q}'),

Y ahora las reglas propiamente dichas. Si su frase no se traduce correctamente, puede intentar aumentar estas reglas para manejar su frase.

In [68]:
reglas = [
    Rule('{P} ⇒ {Q}',         'if {P} then {Q}', 'if {P}, {Q}'),
    Rule('{P} ⋁ {Q}',          'either {P} or else {Q}', 'either {P} or {Q}'),
    Rule('{P} ⋀ {Q}',          'both {P} and {Q}'),
    Rule('～{P} ⋀ ～{Q}',       'neither {P} nor {Q}'),
    Rule('～{A}{P} ⋀ ～{A}{Q}', '{A} neither {P} nor {Q}'), # El Kaiser ni ...
    Rule('～{Q} ⇒ {P}',        '{P} unless {Q}'),
    Rule('{P} ⇒ {Q}',          '{Q} provided that {P}', '{Q} whenever {P}',
                               '{P} implies {Q}', '{P} therefore {Q}',
                               '{Q}, if {P}', '{Q} if {P}', '{P} only if {Q}'),
    Rule('{P} ⋀ {Q}',          '{P} and {Q}', '{P} but {Q}'),
    Rule('{P} ⋁ {Q}',          '{P} or else {Q}', '{P} or {Q}'),
    ]

negaciones = [
    (palabra("not"), ""),
    (palabra("cannot"), "can"),
    (palabra("can't"), "can"),
    (palabra("won't"), "will"),
    (palabra("ain't"), "is"),
    ("n't", ""), # coincidencias como parte de una palabra: didn't, couldn't, etc.
    ]

Ahora el mecanismo para procesar estas reglas. La función clave es `regla_coincidencia`, que compara una frase en inglés con una regla. La función devuelve dos valores, una cadena que representa la traducción de la frase inglesa a la lógica, y `diccionario`, que es un diccionario de pares `{Variable: `value`}` . Si `regla_coincidenciae` encuentra que la regla coincide, llama recursivamente a `reglas_coincidencia` para que coincida con cada uno de los subgrupos de la expresión regular (la `P` y `Q` en `si {P}, entonces {Q}`).
La función `match_literal` se encarga de las negaciones, y es donde el `diccionario` se actualiza.

In [69]:
def reglas_coincidencia(enunciado, reglas, diccionario):
    """Compara la frase con todas las reglas, aceptando la primera coincidencia; o bien la convierte en un átomo.
    Devuelve dos valores: la traducción lógica y un dict de definiciones {P: 'english'}."""
    enunciado = limpiar(enunciado)
    for regla in reglas:
        result = regla_coincidencia(enunciado, regla, diccionario)
        if result:
            return result
    return match_literal(enunciado, negaciones, diccionario)

def regla_coincidencia(enunciado, regla, diccionario):
    "Regla de coincidencia, que devuelve la traducción lógica y el dict de definiciones si la coincidencia tiene éxito."
    output, patterns = regla
    for pat in patterns:
        match = re.match(pat, enunciado, flags=re.I)
        if match:
            groups = match.groupdict()
            for P in sorted(groups): # Aplicar recursivamente reglas a cada uno de los grupos coincidentes
                groups[P] = reglas_coincidencia(groups[P], reglas, diccionario)[0]
            return '(' + output.format(**groups) + ')', diccionario

def match_literal(enunciado, negaciones, diccionario):
    "Ninguna regla coincide; la frase es un átomo. Añadir nueva proposición a diccionario. Manejar la negación."
    polarity = ''
    for (neg, pos) in negaciones:
        (enunciado, n) = re.subn(neg, pos, enunciado, flags=re.I)
        polarity += n * '～'
    enunciado = limpiar(enunciado)
    P = nombre_proposicion(enunciado, diccionario)
    diccionario[P] = enunciado
    return polarity + P, diccionario

def nombre_proposicion(enunciado, diccionario, names='PQRSTUVWXYZBCDEFGHJKLMN'):
    "Devuelve el nombre antiguo de esta sentencia, si se usaba antes, o un nombre nuevo sin usar."
    inverso = {diccionario[P]: P for P in diccionario}
    if enunciado in inverso:
        return inverso[enunciado]                      # Buscar nombre usado anteriormente
    else:
        return next(P for P in names if P not in diccionario) # Utilizar un nuevo nombre sin usar

def limpiar(text):
    "Eliminar los espacios en blanco redundantes; tratar el apóstrofe y la coma/punto final."
    return ' '.join(text.split()).replace("’", "'").rstrip('.').rstrip(',')

Por ejemplo:

In [ ]:
regla_coincidencia("If loving you is wrong, I don't want to be right",
           Rule('{P} ⇒ {Q}', 'if {P}, {Q}'),
           {})

Aquí hay algunos enunciados de prueba más, y una función de nivel superior para manejarlas:

In [ ]:
enunciados = '''
Polkadots and Moonbeams.
If you liked it then you shoulda put a ring on it.
If you build it, he will come.
It don't mean a thing, if it ain't got that swing.
If loving you is wrong, I don't want to be right.
Should I stay or should I go.
I shouldn't go and I shouldn't not go.
If I fell in love with you,
  would you promise to be true
  and help me understand.
I could while away the hours
  conferrin' with the flowers,
  consulting with the rain
  and my head I'd be a scratchin'
  while my thoughts are busy hatchin'
  if I only had a brain.
There's a federal tax, and a state tax, and a city tax, and a street tax, and a sewer tax.
A ham sandwich is better than nothing
  and nothing is better than eternal happiness
  therefore a ham sandwich is better than eternal happiness.
If I were a carpenter
  and you were a lady,
  would you marry me anyway?
  and would you have my baby.
Either Danny didn't come to the party or Virgil didn't come to the party.
Either Wotan will triumph and Valhalla will be saved or else he won't and Alberic will have
  the final word.
Sieglinde will survive, and either her son will gain the Ring and Wotan’s plan
  will be fulfilled or else Valhalla will be destroyed.
Wotan will intervene and cause Siegmund's death unless either Fricka relents
  or Brunnhilde has her way.
Figaro and Susanna will wed provided that either Antonio or Figaro pays and Bartolo is satisfied
  or else Marcellina’s contract is voided and the Countess does not act rashly.
If the Kaiser neither prevents Bismarck from resigning nor supports the Liberals,
  then the military will be in control and either Moltke's plan will be executed
  or else the people will revolt and the Reich will not survive'''.split('.')

import textwrap

def logica(enunciados, width=80):
    "Coteja las reglas con cada enunciado del texto y muestra cada resultado."
    for s in map(limpiar, enunciados):
        logica, diccionario = reglas_coincidencia(s, reglas, {})
        print('\n' + textwrap.fill('Ingles: ' + s +'.', width), '\n\nProposicion Logica:', logica)
        for P in sorted(diccionario):
            print('{}: {}'.format(P, diccionario[P]))

logica(enunciados)

Se ve muy bien. Pero lejos de ser perfecto.  Aquí hay algunos errores:

* `Should I stay` *etc.*:<br>las preguntas no son enunciados proposicionales.

* `If I were a carpenter`:<br>no maneja la lógica modal.

* `nothing is better`:<br>no maneja cuantificadores.

* `Either Wotan will triumph and Valhalla will be saved or else he won't`:<br>obtiene `'he will'` como una de las proposiciones, pero mejor sería que se refiriera a `'Wotan will triumph'`.

* `Wotan will intervene and cause Siegmund's death`:<br>consigue `"cause Siegmund's death"` como propuesta, pero mejor sería `"Wotan will cause Siegmund's death"`.

* `Figaro and Susanna will wed`:<br>consigue `"Figaro"` and `"Susanna will wed"` as two separate propositions; this should really be one proposition.

* `"either Antonio or Figaro pays"`:<br>gets `"Antonio"` como una proposición, pero debería ser `"Antonio pays"`.

* `If the Kaiser neither prevents`:<br>utiliza las proposiciones un tanto falsas `PQ` and `PR`. Esto debería hacerse de forma más limpia. El problema es el mismo que el problema anterior con Antonio: No tengo una buena manera de unir el sujeto de una frase verbal a las múltiples partes del verbo/objeto, cuando hay múltiples partes.



De seguro de que más frases de prueba podrian mostrar muchos más tipos de errores, el reto es encontrarlo.